In [10]:
import pandas as pd

# НАЗВАНИЕ ФАЙЛА CSV:
# ============================================================
file_name = 'rls_.csv'
df = pd.read_csv(file_name, sep=';', encoding='cp1251')
# ============================================================


import numpy as np
import string
import nltk
import pickle 

from nltk.stem.snowball import SnowballStemmer 
from tqdm.auto import tqdm, trange
from nltk.stem import *
from nltk.corpus import stopwords
# from pymystem3 import Mystem
from string import punctuation
from nltk import word_tokenize

import re
import tensorflow as tf

from itertools import chain 


def remove_punctuation(text):
    return "".join([ch if ch not in string.punctuation else ' ' for ch in text])

def remove_numbers(text):
    return ''.join([i if not i.isdigit() else ' ' for i in text])

def remove_multiple_spaces(text):
	return re.sub(r'\s+', ' ', text, flags=re.I)

def remove_stop_words(text):
    tokens = word_tokenize(text) 
    tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
    return " ".join(tokens)

def all_preprocessing(df: pd.DataFrame):

    preproccessing = lambda text: (remove_multiple_spaces(remove_numbers(remove_punctuation(text))))
    df['preproccessed'] = list(map(preproccessing, df['Исходники']))
    prep_text = [remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower()))) for text in tqdm(df['Исходники'])]

    df['prep_text'] = prep_text

    stemmer = SnowballStemmer("russian") 
    russian_stopwords = stopwords.words("russian")
    russian_stopwords.extend(['…', '«', '»', '...', 'т.д.', 'т', 'д'])
    text = df['prep_text'][0]

    stemmed_texts_list = []
    for text in tqdm(df['prep_text']):
        tokens = word_tokenize(text)    
        stemmed_tokens = [stemmer.stem(token) for token in tokens if token not in russian_stopwords]
        text = " ".join(stemmed_tokens)
        stemmed_texts_list.append(text)

    df['text_stem'] = stemmed_texts_list
    
    sw_texts_list = []
    for text in tqdm(df['prep_text']):
        tokens = word_tokenize(text)    
        tokens = [token for token in tokens if token not in russian_stopwords and token != ' ']
        text = " ".join(tokens)
        sw_texts_list.append(text)
    
    # Lemmatized column only: 
    return sw_texts_list


# Load tokenizer: 
with open('bow_model_short/tokenizer.pickle', 'rb') as handle:
    tokenize = pickle.load(handle)

# Load models: 
model = tf.keras.models.load_model('bow_model_short')
with open('bow_model_short/addition_model.pickle', 'rb') as m:
    nb = pickle.load(m)

preprocessed_input = all_preprocessing(df) # list
initials_list, result_list = [], []
length = len(preprocessed_input)

for num in range(length):
    if isinstance(preprocessed_input, pd.DataFrame): 
        if not (num in preprocessed_input.index): 
            continue
        row_0 = preprocessed_input[preprocessed_input.index == num]
    elif isinstance(preprocessed_input, list): 
        row_0 = [preprocessed_input[num]]
    elif isinstance(preprocessed_input, pd.Series): 
        if not (num in preprocessed_input.index): 
            continue
        row_0 = [preprocessed_input[num]]
    row_tokenized = tokenize.texts_to_matrix(row_0)
    class_prediction = []
    count = 5
    
    while (len(class_prediction) == 0) & (count > 0):  
        preds_0 = list(chain(*model.predict(row_tokenized).tolist()))
        preds_f = [round(cls) for cls in preds_0]
        class_prediction = [ind+1 for (ind, obj) in enumerate(preds_f) if obj == 1]
        count -= 1
    if len(class_prediction): 
        class_prediction = class_prediction[0]
    else: 
        print('NB model prediction')
        class_prediction = nb.predict(row_0)[0]
    initial = df[df.index == num]['Класс'].values

    initials_list.append(row_0)
    result_list.append(class_prediction)


result = pd.DataFrame({
    'Исходники': initials_list, 
    'Предсказанный класс': result_list
})

result.to_csv('Результат.csv')

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

1/1 [==============================] - 0s 17ms/step


In [8]:
accur

0.3802083333333333